In [1]:
import csv

In [2]:
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import preprocessing as pre
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'sklearn'

In [3]:
import pandas as pd
data = data = pd.read_csv('Getting to Know/Combined-780 - Sheet1.csv')

In [4]:
data

,Signal,Mean (µV),RMS (µV),MRS (µV),Zero Crossings,Frequency (Hz),Delta (µV² /Hz),Theta (µV² /Hz),Alpha (µV² /Hz),Beta (µV² /Hz),Gamma (µV² /Hz),Label
0,e1.1,-111.543263,165.326922,137.464510,6,3.0,13.871195,0.216515,0.157400,0.471267,0.360638,0
1,e1.2,-85.046038,140.633143,111.886389,6,3.0,5.126801,0.176441,0.151454,0.471898,0.364846,0
2,e1.3,-46.838299,126.812428,85.819704,16,8.0,1.261876,0.181805,0.161871,0.472354,0.372668,0
3,e1.4,-47.013536,143.652350,91.850550,7,3.5,0.899144,0.212408,0.298429,0.738894,0.501854,0
4,e1.5,-15.339653,113.011520,53.273050,24,12.0,0.423585,0.195163,0.150992,0.484105,0.371071,0
...,...,...,...,...,...,...,...,...,...,...,...,...
775,e47.1.16,-492.032900,791.292300,693.099200,3,1.5,34.918747,2.114091,0.244951,0.139495,0.062960,1
776,e47.1.17,-255.891500,455.610800,378.904600,4,2.0,15.347951,1.263933,0.188982,0.112097,0.051938,1
777,e47.1.18,224.316200,619.768200,541.530000,8,4.0,24.577597,0.953114,0.249922,0.139794,0.061432,1
778,e47.1.19,-164.945500,986.025200,798.400200,4,2.0,31.002390,1.905827,0.262337,0.121908,0.053995,1


In [4]:
X = data.drop(columns=['Signal','Label'])
y = data['Label']

In [9]:
with open('best-accuracy.csv', 'w') as file:
    writer = csv.writer(file)
    k_values = [4,5,6,7,8,10]
    for row in k_values:
        kf = KFold(n_splits=row)
        row_to_add = []
        dic = {} # to hold accuracies across folds
        for i in range(1,7):
            dic["model{}".format(i)] = []
            
        for train_index, test_index in kf.split(X,y):
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y.loc[train_index], y.loc[test_index]
            X_train_scaled = pre.scale(X_train)
            X_test_scaled = pre.scale(X_test) 
            models = {}
            models["model1"] = svm.SVC(kernel='linear',C=20).fit(X_train_scaled, y_train)
            models["model2"] = KNeighborsClassifier(n_neighbors=5).fit(X_train_scaled, y_train)
            models["model3"] = KNeighborsClassifier(n_neighbors=10).fit(X_train_scaled, y_train)
            models["model4"] = LogisticRegression(random_state=0,max_iter=1000).fit(X_train_scaled, y_train)
            models["model5"] = tree.DecisionTreeClassifier().fit(X_train_scaled, y_train)
            models["model6"] = MLPClassifier(random_state=0, max_iter=5000, alpha=0.001, hidden_layer_sizes=(10,2)).fit(X_train_scaled, y_train)
            for i in range(1,7):
                y_pred = models["model{}".format(i)].predict(X_test_scaled)
                cm = confusion_matrix(y_test, y_pred)
                total = sum(sum(cm))
                acc =(cm[0,0]+cm[1,1])/total
                dic.get("model{}".format(i)).append(acc)
                
        for i in range(1,7):
            list_of_acc = dic.get("model{}".format(i))
            row_to_add.append(max(list_of_acc))
            print(list_of_acc)
        writer.writerow(row_to_add)
        print("\n")

[0.64102564102564108, 0.46153846153846156, 0.65641025641025641, 0.64615384615384619]
[0.36410256410256409, 0.47179487179487178, 0.5641025641025641, 0.61538461538461542]
[0.38461538461538464, 0.48205128205128206, 0.54358974358974355, 0.61538461538461542]
[0.60512820512820509, 0.39487179487179486, 0.60512820512820509, 0.6512820512820513]
[0.26666666666666666, 0.48205128205128206, 0.44615384615384618, 0.55897435897435899]
[0.50769230769230766, 0.42051282051282052, 0.58461538461538465, 0.61025641025641031]


[0.58333333333333337, 0.25, 0.41025641025641024, 0.78846153846153844, 0.66666666666666663]
[0.28205128205128205, 0.47435897435897434, 0.51282051282051277, 0.71794871794871795, 0.60256410256410253]
[0.32051282051282054, 0.45512820512820512, 0.46153846153846156, 0.73076923076923073, 0.64743589743589747]
[0.53846153846153844, 0.33974358974358976, 0.38461538461538464, 0.71153846153846156, 0.6858974358974359]
[0.40384615384615385, 0.53205128205128205, 0.5641025641025641, 0.65384615384615385